# Batch-send emails using a csv table

In [2]:
import smtplib, email, ssl
import pandas as pd
import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from bs4 import BeautifulSoup as bs
from email.utils import formatdate
from os.path import basename # extract basename for the file path
from dotenv import load_dotenv
import os
import boto3
from botocore.exceptions import ClientError

# Connver to Email Server

## Zoho
NOT recomended due to limits of email rates. You can blocked pretty frequently if too many bounces or sending out too many emails in a short period of time.

In [20]:
load_dotenv()

EMAIL_HOST = os.environ.get('EMAIL_HOST')
EMAIL_PORT = os.environ.get('EMAIL_PORT')

# volunteer
# EMAIL_HOST_PASSWORD =os.environ.get('EMAIL_HOST_PASSWORD')
# EMAIL_HOST_USER = os.environ.get('EMAIL_HOST_USER')
#management
EMAIL_HOST_PASSWORD =os.environ.get('EMAIL_HOST_PASSWORD_MANAGEMENT')
EMAIL_HOST_USER = os.environ.get('EMAIL_HOST_USER_MANAGEMENT')

## AWS SES
Recomended for production

See code example [here](https://docs.aws.amazon.com/ses/latest/DeveloperGuide/examples.html)

In [10]:
# Replace sender@example.com with your "From" address.
# This address must be verified with Amazon SES.(voluntneer, no-reply)
SENDER = "Chinese Antibody Society <volunteer@chineseantibody.org>"

# Replace recipient@example.com with a "To" address. If your account
# is still in the sandbox, this address must be verified.
RECIPIENT = ["gymchickyu@gmail.com"]

# If necessary, replace us-west-2 with the AWS Region you're using for Amazon SES.
AWS_REGION = "us-east-1"

# The subject line for the email.
SUBJECT = "Amazon SES Test (SDK for Python)"

# The email body for recipients with non-HTML email clients. Neglectible
BODY_TEXT = ("Amazon SES Test (Python)\r\n"
             "This email was sent with Amazon SES using the "
             "AWS SDK for Python (Boto)."
            )

# The HTML body of the email.
BODY_HTML = """<html>
<head></head>
<body>
  <h1>Amazon SES Test (SDK for Python)</h1>
  <p>This email was sent with
    <a href='https://aws.amazon.com/ses/'>Amazon SES</a> using the
    <a href='https://aws.amazon.com/sdk-for-python/'>
      AWS SDK for Python (Boto)</a>.</p>
</body>
</html>
            """

# The character encoding for the email.
CHARSET = "UTF-8"

# Create a new SES resource and specify a region.
client = boto3.client(
    'ses',
    region_name='us-east-1'
)

# Try to send the email.
try:
    #Provide the contents of the email.
    response = client.send_email(
        Destination={
            'ToAddresses': RECIPIENT,
        },
        Message={
            'Body': {
                'Html': {
                    'Charset': CHARSET,
                    'Data': BODY_HTML,
                },
                'Text': {
                    'Charset': CHARSET,
                    'Data': BODY_TEXT,
                },
            },
            'Subject': {
                'Charset': CHARSET,
                'Data': SUBJECT,
            },
        },
        Source=SENDER,
        # If you are not using a configuration set, comment or delete the
        # following line
        # ConfigurationSetName=CONFIGURATION_SET,
    )
# Display an error if something goes wrong.
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    print("Email sent! Message ID:"),
    print(response['MessageId'])

Email sent! Message ID:
01000179b9e892bd-2d36812d-2533-4043-a3a9-992b75007ab5-000000


# Send email with attachments

In [9]:
# email contents
subject = '华人抗体协会恭贺您新春快乐'
date = datetime.date.strftime(datetime.date.today(), '%Y-%m-%d')


# Load dataset
df = pd.read_csv('example/test.csv')[:1] # load 40 rows each time is OK. Sometimes server disconnects but you can pick up from where you're.

df


,Unique ID,First Name,Last Name,Name,Start,Status,End,Email
0,Volun0001,Jane,Doe,Jane Doe,2019,Active,2020,youremail@gmail.com


In [10]:
for id, name, email in zip(df['Unique ID'], df['Name'], df['Email']):

    # get file name and path
    fn = id + ' - ' + name +'.pdf'
    fp = 'example/test/' + fn

    # message
    message = '尊敬的'+name+ """:<br><br>
希望你一切安好。2020年就要过去了。虽然有疫情的挑战，协会今年依然有长足的进展。而这一切与你的大力支持密不可分。为了感谢你对协会的付出，协会制作了志愿者证书以聊表寸心。<u>证书请见附件</u>。 祝你在2021年万事如意! <br><br>
华人抗体协会<br>""" + date

    # Parse the html content into soup object
    with open('template/2021-new-year.html') as temp:
        soup=bs(temp)

        # add message to soup
        soup.find("p", id="message").append(message)
        soup=soup.prettify(formatter=None)
        soup = soup.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace("\\'", " \' ").replace('\'re', " are" )

    # message
    msg = MIMEMultipart() # must reinitialize for each email
    msg.set_charset('utf8')
    msg['From'] = EMAIL_HOST_USER
    msg['Subject'] = subject
    msg['Date'] = formatdate(localtime=True)
    msg['To'] = email
    msg.attach(MIMEText(str(soup), 'html'))

    # attach
    with open(fp, "rb") as fil:
        part = MIMEApplication(
            fil.read(),
            Name=basename(fp)
        )
    part['Content-Disposition'] = 'attachment; filename="%s"' % basename(fp)
    msg.attach(part)

    # send
    SERVER = smtplib.SMTP(EMAIL_HOST, EMAIL_PORT)
    SERVER.connect(EMAIL_HOST, EMAIL_PORT)
    SERVER.starttls()
    SERVER.login(EMAIL_HOST_USER, EMAIL_HOST_PASSWORD)
    SERVER.sendmail(
            EMAIL_HOST_USER,
            email,
            msg.as_string()
        )
    print('Email with attachment '+ fn + ' sent to ' + email) # print this, in case server disconnects and you can resume from where it is left.

Email with attachment Volun0001 - Jane Doe.pdf sent to youremail@gmail.com


# Send pure html email

In [ ]:
# email contents
subject = '华人抗体协会恭贺您新春快乐!'
date = datetime.date.strftime(datetime.date.today(), '%Y-%m-%d')

# Load dataset
df = pd.read_csv('cas-data/020921-update contact list-p2.csv')# load 40 rows each time is OK. Sometimes server disconnects but you can pick up from where you're.

df

In [ ]:
for name, email in zip(df['Name'], df['Email']):

    # message
    message = '尊敬的 Dr.'+name+ """<br>感谢您对华人抗体协会一如既往地支持和鼓励。值此新春佳节，祝您和家人春节快乐，身体健康，万事如意！<br><br>此致<br>华人抗体协会<br>2021年农历新年<br>
    """ + date

    # Parse the html content into soup object
    with open('template/2021-new-year.html') as temp:
        soup=bs(temp)

        # add message to soup
        soup.find("p", id="message").append(message)
        soup=soup.prettify(formatter=None)
        soup = soup.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace("\\'", " \' ").replace('\'re', " are" )

        # message
        msg = MIMEMultipart() # must reinitialize for each email
        msg.set_charset('utf8')
        msg['From'] = EMAIL_HOST_USER
        msg['Subject'] = subject
        msg['Date'] = formatdate(localtime=True)
        msg['To'] = email
        msg.attach(MIMEText(str(soup), 'html'))

        # send
        SERVER = smtplib.SMTP(EMAIL_HOST, EMAIL_PORT)
        SERVER.connect(EMAIL_HOST, EMAIL_PORT)
        SERVER.starttls()
        SERVER.login(EMAIL_HOST_USER, EMAIL_HOST_PASSWORD)
        SERVER.sendmail(
                EMAIL_HOST_USER,
                email,
                msg.as_string()
            )
        print('Email sent to ' + email) # print this, in case server disconnects and you can resume from where it is left.